In [1]:
import numpy as np
import pandas as pd
import datetime
import os

In [193]:
# def process(file):
    
#     temp = pd.read_csv(file)
#     temp = temp[temp['Country/Region'] == 'US'] if 'Country/Region' in temp.columns else temp[temp['Country_Region'] == 'US']
    
#     if 'Admin2' in temp.columns:
#         temp['County'] = temp.Admin2
#         temp['State'] = temp.Province_State
#     elif temp['Province/State'].str.contains(',').any() and (temp['Province/State'] != 'Virgin Islands, U.S.').all():
#         x = temp['Province/State'].str.replace(' (From Diamond Princess)','', regex=False).replace('Unassigned Location','', regex=False).str.split(', ')
#         temp['County'] = x.map(lambda x: x[0])
#         temp['State'] = x.map(lambda x: x[-1])
#     else:
#         temp['County'] = ''
#         temp['State'] = temp['Province/State'].str.replace(' (From Diamond Princess)','', regex=False).str.replace('Unassigned Location','', regex=False)
    
#     temp['County'] = temp['County'].str.replace(' County', '', regex=False)
#     temp['Date'] = pd.to_datetime(file[-14:-4]).strftime('%Y-%m-%d')
    
#     temp['Lat'] = temp['Latitude'] if 'Latitude' in temp.columns else np.nan
    
#     if 'Longitude' in temp.columns:
#         temp['Long'] = temp['Longitude']
#     elif 'Long_' in temp.columns:
#         temp['Long'] = temp['Long_']
#     else:
#         temp['Long'] = np.nan
    
#     temp['Deaths'] = temp.Deaths
#     temp['Recovered'] = temp.Recovered if 'Recovered' in temp.columns else 0
#     temp['Active'] = temp['Active'] if 'Active' in temp.columns else 0
#     temp['FIPS'] = temp['FIPS'] if 'FIPS' in temp.columns else np.nan
    
#     temp = temp[['FIPS', 'County', 'State', 'Date', 'Lat', 'Long', 'Confirmed', 'Deaths', 'Recovered', 'Active']]        

#     return temp

In [337]:
# path = '../../../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'

# if not os.path.exists('covid_19_US.csv'):
#     files = [i for i in os.listdir(path) if 'csv' in i]
#     data = pd.DataFrame([])

#     for i in files:
#         print('loading {}'.format(i))
#         temp = process(path + i)
#         data = pd.concat([data, temp], ignore_index=True)

#     print(data.shape)
# else:
#     data = pd.read_csv('covid_19_US.csv').drop('Unnamed: 0', axis=1)
#     files = [i for i in os.listdir(path) if i[:5] > max(data.Date)[-5:] and '2020' in i]
    
#     for i in files:
#         print('loading {}'.format(i))

#         try:
#             temp = process(path + i)
#         except ValueError:
#             print('New file has different format.')
        
#         assert(data.shape[1] == 10)
#         data = pd.concat([data, temp], ignore_index=True)

# print('There are total {} records.'.format(data.shape[0]))

# data.sort_values('Date').to_csv('covid_19_US.csv')

In [119]:
data.sample(10)

FIPS            County         State        Date      Lat  \
23820  38065.0            Oliver  North Dakota  2020-03-29      NaN   
21378  47107.0            McMinn     Tennessee  2020-03-28      NaN   
31099  35047.0        San Miguel    New Mexico  2020-03-31      NaN   
44376  51740.0        Portsmouth      Virginia  2020-04-06      NaN   
831        0.0  Riverside County            CA  2020-03-08  33.9533   
29608  38043.0            Kidder  North Dakota  2020-03-30      NaN   
16590  16011.0           Bingham         Idaho  2020-03-26      NaN   
2853   38003.0            Barnes  North Dakota  2020-03-22      NaN   
5996   47153.0        Sequatchie     Tennessee  2020-03-23      NaN   
6799   48027.0              Bell         Texas  2020-03-23      NaN   

             Long  Confirmed  Deaths  Recovered  Active  
23820 -101.340616        0.0     0.0        0.0       0  
21378  -84.615207        3.0     0.0        0.0       0  
31099 -104.816356        1.0     0.0        0.0       0  
44376  -76.359716       27.0     0.0        0.0       0  
831   -117.396100        1.0     0.0        0.0       0  
29608  -99.775079        0.0     0.0        0.0       0  
16590 -112.397844        2.0     0.0        0.0       0  
2853   -98.066060        0.0     0.0        0.0       0  
5996   -85.410221        0.0     0.0        0.0       0  
6799   -97.478503       16.0     0.0        0.0       0

In [561]:
def ETL(file):

    # loading COVID-19 data and zipcode
    print('Processing {}'.format(file[-14:]))
    data = pd.read_csv(file)
    zipinfo = pd.read_csv('uszips.csv')[['county_fips', 'county_name', 'state_name', 'lat', 'lng']].drop_duplicates()
    zipinfo = zipinfo.groupby(['county_fips', 'county_name', 'state_name']).agg('mean').reset_index()

    # extract US records and features
    data = data.loc[data.Country_Region == 'US', ['FIPS', 'Admin2', 'Province_State', 'Confirmed', 'Deaths']]

    # rename features
    data.columns = ['FIPS', 'County', 'State', 'Confirmed', 'Deaths']

    # remove 'City' in the County columns to match zipcode table
    data.County = data.County.str.replace(' City', '', regex=False)

    # final output features
    feature = ['FIPS', 'County', 'State', 'lat', 'lng', 'Confirmed', 'Deaths']

    # extract non-missing records
    x = pd.merge(data, zipinfo, left_on = ['FIPS', 'County', 'State'],
                 right_on = ['county_fips', 'county_name', 'state_name'], how = 'inner')[feature]

    # correct wrong FIPS and fill in missing
    y = data[~data.FIPS.isin(x.FIPS)]
    z = pd.merge(y, zipinfo, left_on = ['County', 'State'], right_on = ['county_name', 'state_name'], how = 'inner')
    z.FIPS = z.county_fips
    z = z[feature]

    # merge two tables
    data = pd.concat([x, z], ignore_index = True).groupby(['FIPS', 'County', 'State']).agg('sum').reset_index()
    
    return data

def increment(x, y):
    temp = pd.merge(x, y[['FIPS', 'County', 'State', 'Confirmed', 'Deaths']],
                    on=['FIPS', 'County', 'State'], how='left')
    temp.Confirmed_y.fillna(0, inplace=True)
    temp.Deaths_y.fillna(0, inplace=True)
    temp['Confirmed_new'] = (temp.Confirmed_x - temp.Confirmed_y).astype(int)
    temp['Deaths_new'] = (temp.Deaths_x - temp.Deaths_y).astype(int)
    temp['Confirmed'] = temp.Confirmed_x
    temp['Deaths'] = temp.Deaths_x
    temp = temp[['FIPS', 'County', 'State', 'lat', 'lng', 'Confirmed', 'Confirmed_new', 'Deaths', 'Deaths_new']]

    return temp

In [562]:
path = '../../../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
file = path + '04-10-2020.csv'
data10 = ETL(file)

Processing 04-10-2020.csv


In [563]:
path = '../../../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
file = path + '04-11-2020.csv'
data11 = ETL(file)

Processing 04-11-2020.csv


In [565]:
diff = increment(data11,data10)
diff.shape

(2632, 9)

In [532]:
pop = pd.read_csv('us_county.csv')[['fips', 'population']]
temp = pd.merge(diff, pop, left_on = 'FIPS', right_on = 'fips', how = 'left').drop('fips', axis = 1)
temp.eval("Confirmed_rate = Confirmed / population", inplace =True)
temp.eval("Deaths_rate = Deaths / Confirmed", inplace =True)

In [535]:
temp.sort_values('Confirmed_rate', ascending=False).head()

FIPS       County     State        lat         lng  Confirmed  \
1547  36061.0     New York  New York  40.762071  -73.976000      98308   
1558  36087.0     Rockland  New York  41.128793  -74.003296       7477   
513   16013.0       Blaine     Idaho  43.537407 -114.256719        452   
1574  36119.0  Westchester  New York  41.114868  -73.782423      18729   
1546  36059.0       Nassau  New York  40.730876  -73.614145      22584   

      Confirmed_new  Deaths  Deaths_new  population  Confirmed_rate  \
1547         5924.0    6367       547.0     1632480        0.060220   
1558          355.0     200         0.0      323686        0.023100   
513             6.0       5         0.0       21994        0.020551   
1574          652.0     461        72.0      968815        0.019332   
1546         1072.0     792        69.0     1356564        0.016648   

      Deaths_rate  
1547     0.064766  
1558     0.026749  
513      0.011062  
1574     0.024614  
1546     0.035069

In [528]:
path = '../../../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
file = path + '04-10-2020.csv'
data10 = ETL(file)

Processing 04-10-2020.csv


489630

In [2]:
x = pd.read_csv('./uszips.csv')[['county_fips', 'county_name', 'state_name', 'lat', 'lng']].drop_duplicates()
x.groupby(['county_fips', 'county_name', 'state_name']).agg('mean').reset_index().head()

county_fips county_name state_name        lat        lng
0         1001     Autauga    Alabama  32.580766 -86.615497
1         1003     Baldwin    Alabama  30.594986 -87.724403
2         1005     Barbour    Alabama  31.780406 -85.435020
3         1007        Bibb    Alabama  33.020930 -87.090535
4         1009      Blount    Alabama  33.924849 -86.599899

In [3]:
pd.read_csv('./uszips.csv').groupby(['county_fips', 'county_name', 'state_name'])['lat', 'lng'].agg('mean').head()

lat        lng
county_fips county_name state_name                      
1001        Autauga     Alabama     32.580766 -86.615497
1003        Baldwin     Alabama     30.594986 -87.724403
1005        Barbour     Alabama     31.780406 -85.435020
1007        Bibb        Alabama     33.020930 -87.090535
1009        Blount      Alabama     33.924849 -86.599899